In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
from config import gkey

In [2]:
# Questionaire: Home address, type of food, and radius. 

home_address = input(f"What is your home address?(Please separate with space, with no comma): ")
yes_no = input(f"Is {home_address} the correct address? (YES or NO): ")
while yes_no == "NO":
    home_address = input(f"What is your home address?: ")
    yes_no = input(f"Is {home_address} the correct address? (YES or NO): ")
    
type_food = input(f"What type of food are you looking for?")
yes_no2 = input(f"Is {type_food} food what you are looking for? (YES or NO): ")
while yes_no2 == "NO":
    type_food = input(f"What type of food are you looking for?")
    yes_no2 = input(f"Is {type_food} food what you are looking for? (YES or NO): ")
    
radius = input(f"What would be your acceptable radius in meters?: ")
yes_no3 = input(f"Is {radius} meters acceptable to you? ")
while yes_no3 == "NO":
    radius = input(f"What would be your acceptable radius in meters?: ")
    yes_no3 = input(f"Is {radius} meters acceptable to you? (YES or NO) ")


What is your home address?(Please separate with space, with no comma):  37967 Starflower St, Newark, CA 94560
Is 37967 Starflower St, Newark, CA 94560 the correct address? (YES or NO):  YES
What type of food are you looking for? ramen
Is ramen food what you are looking for? (YES or NO):  YES
What would be your acceptable radius in meters?:  10000
Is 10000 meters acceptable to you?  YES


In [7]:
# Utilizing Google Geocoding API to convert addresses to geographic coordinations in the format of (latitude, longitudee)

# formating the address to the correct format
adj_home_address = home_address.replace(" ", "+")

# Setting parameter
params = {
    "address": adj_home_address,
    "key": gkey
}

# Set up URL and request get
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
response = requests.get(base_url, params).json()

# Assigning latitude, and longitude to variables
lat = response["results"][0]["geometry"]["bounds"]["northeast"]["lat"]
lng = response["results"][0]["geometry"]["bounds"]["northeast"]["lng"]

In [8]:
# Utilizing Google Nearby Search API to search for nearby restaurants within certain range to home address. 

# Creating a blank dataframe with columns.
df = pd.DataFrame()
df["Restaurant Name"] = ""
df["Restaurant Rating"] = ""
df["Restaurant Price Level"] = ""
df["Restaurant Total Reviews"] = ""
df["Restaurant Address"] = ""

# set coordinate to the format: lat,lng
geocode = f"{lat},{lng}"

# Set parameters
params = {
    "location":geocode,
    "radius": radius,
    "keyword": type_food,
    "type": "restaurant",
    "key": gkey,
}

# Set up URL, and request get
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
response = requests.get(base_url, params).json()


#name = response["results"][0]["name"]
#rating = response["results"][0]["rating"]
#price_lv = response["results"][0]["price_level"]
#user_ratings_total = response["results"][0]["user_ratings_total"]

# Pulling relevant information and adding to our dataframe.
for i in range(len(response["results"][0])):
    
    try:
        name = response["results"][i]["name"]
        rating = response["results"][i]["rating"]
        price_lv = response["results"][i]["price_level"]
        user_ratings_total = response["results"][i]["user_ratings_total"]
        address = response["results"][i]["vicinity"]

        df.loc[i, "Restaurant Name"] = name
        df.loc[i, "Restaurant Rating"] = rating
        df.loc[i, "Restaurant Price Level"] = price_lv
        df.loc[i, "Restaurant Total Reviews"] = user_ratings_total
        df.loc[i, "Restaurant Address"] = address
        
    except (KeyError, IndexError):
        print(f"Failed - Skipping...")
    
    print("------------")

------------
------------
------------
------------
------------
Failed - Skipping...
------------
------------
------------
Failed - Skipping...
------------
Failed - Skipping...
------------
------------
------------
------------
------------
------------
Failed - Skipping...
------------
Failed - Skipping...
------------


In [9]:
# Output

df.sort_values(by = ["Restaurant Total Reviews", "Restaurant Price Level"], ascending = [False,True], inplace=True)
df.reset_index(drop=True, inplace = True)
df.head()

,Restaurant Name,Restaurant Rating,Restaurant Price Level,Restaurant Total Reviews,Restaurant Address
0,Tadamasa,4.5,1,437,"34672 Alvarado-Niles Rd, Union City"
1,Ajisen Ramen,4.3,1,341,"47890 Warm Springs Blvd, Fremont"
2,Ramen Shu,4.4,2,317,"34308 Alvarado-Niles Rd, Union City"
3,Yugo Ramen,4.4,2,272,"2014 Newpark Mall Rd, Newark"
4,Bodo Lucky Japanese Restaurant,4.4,1,262,"46260 Warm Springs Blvd, Fremont"


In [6]:
# Save csv file.
save_to = f"Resources/{type_food}_restaurants_locator.csv"
df.to_csv(save_to)